# 13장. RDD 고급개념

In [1]:
myCollection = "Spark The Definitive Guide : Big Data Processing Made Simple".split()
words = sc.parallelize(myCollection, 2)
words

ParallelCollectionRDD[0] at parallelize at PythonRDD.scala:194

## 13.1 키-값 형태의 RDD
ByKey 형태의 메서드는 PairRDD 타입만 사용 가능 > 키-값, 튜플 같은

#### keyBy : 현재 값으로부터 키를 생성하는 함수

In [2]:
# word.lower()[0] 이 키가 됨
keyword = words.keyBy(lambda word : word.lower()[0])
keyword.collect()

[('s', 'Spark'),
 ('t', 'The'),
 ('d', 'Definitive'),
 ('g', 'Guide'),
 (':', ':'),
 ('b', 'Big'),
 ('d', 'Data'),
 ('p', 'Processing'),
 ('m', 'Made'),
 ('s', 'Simple')]

#### mapValues : 값을 매핑하는 함수

In [3]:
keyword.mapValues(lambda word : word.upper()).collect()

[('s', 'SPARK'),
 ('t', 'THE'),
 ('d', 'DEFINITIVE'),
 ('g', 'GUIDE'),
 (':', ':'),
 ('b', 'BIG'),
 ('d', 'DATA'),
 ('p', 'PROCESSING'),
 ('m', 'MADE'),
 ('s', 'SIMPLE')]

#### flatMapValues : 리스트 형태를 키값을 유지한채 값들만 flat하게 만들어버리는 함수

In [4]:
keyword.flatMapValues(lambda word : word.upper()).collect()

[('s', 'S'),
 ('s', 'P'),
 ('s', 'A'),
 ('s', 'R'),
 ('s', 'K'),
 ('t', 'T'),
 ('t', 'H'),
 ('t', 'E'),
 ('d', 'D'),
 ('d', 'E'),
 ('d', 'F'),
 ('d', 'I'),
 ('d', 'N'),
 ('d', 'I'),
 ('d', 'T'),
 ('d', 'I'),
 ('d', 'V'),
 ('d', 'E'),
 ('g', 'G'),
 ('g', 'U'),
 ('g', 'I'),
 ('g', 'D'),
 ('g', 'E'),
 (':', ':'),
 ('b', 'B'),
 ('b', 'I'),
 ('b', 'G'),
 ('d', 'D'),
 ('d', 'A'),
 ('d', 'T'),
 ('d', 'A'),
 ('p', 'P'),
 ('p', 'R'),
 ('p', 'O'),
 ('p', 'C'),
 ('p', 'E'),
 ('p', 'S'),
 ('p', 'S'),
 ('p', 'I'),
 ('p', 'N'),
 ('p', 'G'),
 ('m', 'M'),
 ('m', 'A'),
 ('m', 'D'),
 ('m', 'E'),
 ('s', 'S'),
 ('s', 'I'),
 ('s', 'M'),
 ('s', 'P'),
 ('s', 'L'),
 ('s', 'E')]

In [5]:
# 전부다 flat하게 만들어버리는 flatMap()
keyword.flatMap(lambda word : (word[0], word[1].upper())).collect()

['s',
 'SPARK',
 't',
 'THE',
 'd',
 'DEFINITIVE',
 'g',
 'GUIDE',
 ':',
 ':',
 'b',
 'BIG',
 'd',
 'DATA',
 'p',
 'PROCESSING',
 'm',
 'MADE',
 's',
 'SIMPLE']

#### 키,값 추출하기

In [6]:
print(keyword.keys().collect())
keyword.values().collect()

['s', 't', 'd', 'g', ':', 'b', 'd', 'p', 'm', 's']


['Spark',
 'The',
 'Definitive',
 'Guide',
 ':',
 'Big',
 'Data',
 'Processing',
 'Made',
 'Simple']

#### lookup : 특정 키에 대한 결과를 볼수 있는 메서드

In [7]:
keyword.lookup("s")

['Spark', 'Simple']

#### sampleByKey : 근사치나 정확도를 이용해 RDD샘플을 키를 기준으로 추출

In [8]:
import random
distinctChars = words.flatMap(lambda word:list(word.lower())).distinct().collect()
sampleMap = dict(map(lambda c : (c, random.random()), distinctChars))

words.map(lambda word:(word.lower()[0], word)).sampleByKey(True, sampleMap, 1234).collect()

[('t', 'The'), ('d', 'Definitive'), ('d', 'Data'), ('s', 'Simple')]

## 13.2 집계

In [9]:
chars = words.flatMap(lambda word : word.lower())
KVcharacters = chars.map(lambda letter : (letter, 1))

def maxFunc(x, y):
    return max(x, y)
def addFunc(x, y):
    return x + y

nums = sc.parallelize(range(1, 31), 5)

#### countByKey : 키값을 기준으로 레코드 개수를 리턴

In [10]:
KVcharacters.countByKey()

defaultdict(int,
            {':': 1,
             'a': 4,
             'b': 1,
             'c': 1,
             'd': 4,
             'e': 7,
             'f': 1,
             'g': 3,
             'h': 1,
             'i': 7,
             'k': 1,
             'l': 1,
             'm': 2,
             'n': 2,
             'o': 1,
             'p': 3,
             'r': 2,
             's': 4,
             't': 3,
             'u': 1,
             'v': 1})

### 집계 연산 구현 방식 이해
- groupByKey : 키 값으로 묶어 rdd 반환
    - 모든 익스큐터에서 키와 관련된 모든 값을 읽어야하기에 키값별로 편차가 심한 데이터는 OutOfMemoryError가 발생할수 있다

In [11]:
from functools import reduce
KVcharacters.groupByKey().map(lambda row : (row[0], reduce(addFunc, row[1]))).collect()

[('s', 4),
 ('p', 3),
 ('r', 2),
 ('h', 1),
 ('d', 4),
 ('i', 7),
 ('g', 3),
 ('b', 1),
 ('c', 1),
 ('l', 1),
 ('a', 4),
 ('k', 1),
 ('t', 3),
 ('e', 7),
 ('f', 1),
 ('n', 2),
 ('v', 1),
 ('u', 1),
 (':', 1),
 ('o', 1),
 ('m', 2)]

- reduceByKey : 각각의 파티션에서 reduce가 일어나기에 모든값을 메모리에 유지하지 않아도 된다
    - 작업부하도 줄이고 안정성, 연산 수행속도 향상을 기대

In [12]:
KVcharacters.reduceByKey(addFunc).collect()

[('s', 4),
 ('p', 3),
 ('r', 2),
 ('h', 1),
 ('d', 4),
 ('i', 7),
 ('g', 3),
 ('b', 1),
 ('c', 1),
 ('l', 1),
 ('a', 4),
 ('k', 1),
 ('t', 3),
 ('e', 7),
 ('f', 1),
 ('n', 2),
 ('v', 1),
 ('u', 1),
 (':', 1),
 ('o', 1),
 ('m', 2)]

### 기타 집계 메서드

#### aggregate(집계시작값, 파티션내 수행함수, 모든파티션에 수행될 함수)
- aggregate : 드라이버에서 최종집계 >> 익스큐터의 결과가 크면 OutOfMemory
- treeAggregate : 집계처리를 여러단계로 구성 >> 드라이버 메모리 소모를 줄임

In [13]:
nums.aggregate(0, maxFunc, addFunc)

90

In [14]:
depth = 3
nums.treeAggregate(0, maxFunc, addFunc, depth)

90

#### aggregateByKey : 파티션 기준연산이 아니라 키를 기준으로 연산

In [15]:
nums.keyBy(lambda x : 1 if x > 15 else 2).aggregateByKey(0, maxFunc, addFunc).collect()

[(1, 72), (2, 33)]

#### combineByKey : 키를 기준으로 연산을 하며 파라미터로 사용된 함수에 따라 값을 병합
 병합과정에서 타입이 바뀔 수 있음

In [16]:
def valToCombiner(value):
    return [value]
def mergeValuesFunc(vals, valToAppend):
    vals.append(valToAppend)
    return vals
def mergeCombineFunc(vals1, vals2):
    return vals1 + vals2
outputPartitions = 6

KVcharacters.combineByKey(valToCombiner, mergeValuesFunc, mergeCombineFunc, outputPartitions).collect()

[('s', [1, 1, 1, 1]),
 ('d', [1, 1, 1, 1]),
 ('l', [1]),
 ('v', [1]),
 (':', [1]),
 ('p', [1, 1, 1]),
 ('r', [1, 1]),
 ('c', [1]),
 ('k', [1]),
 ('t', [1, 1, 1]),
 ('n', [1, 1]),
 ('u', [1]),
 ('o', [1]),
 ('h', [1]),
 ('i', [1, 1, 1, 1, 1, 1, 1]),
 ('g', [1, 1, 1]),
 ('b', [1]),
 ('a', [1, 1, 1, 1]),
 ('e', [1, 1, 1, 1, 1, 1, 1]),
 ('f', [1]),
 ('m', [1, 1])]

#### foldByKey : 결합함수와 항등원인 0을 이용해 각 키의 값을 병합 
reduceByKey랑 비슷 ; 항등원을 인자로 줘야하는 점이 다름

In [17]:
KVcharacters.foldByKey(0, addFunc).collect()

[('s', 4),
 ('p', 3),
 ('r', 2),
 ('h', 1),
 ('d', 4),
 ('i', 7),
 ('g', 3),
 ('b', 1),
 ('c', 1),
 ('l', 1),
 ('a', 4),
 ('k', 1),
 ('t', 3),
 ('e', 7),
 ('f', 1),
 ('n', 2),
 ('v', 1),
 ('u', 1),
 (':', 1),
 ('o', 1),
 ('m', 2)]

## 13.3 cogroup 
스칼라를 사용하면 3개의 키-값 형태의 RDD를 그룹화
파이썬을 사용하면 2개의 키-값 형태의 RDD를 그룹화
- 키를 기준으로 그룹화!!!
- RDD에 대한 그룹기반의 조인을 수행
- 출력파티션 수나 클러스터에 데이터 분산 방식을 제어하기 위해 사용자 정의 파티션 함수를 파라미터로 줄 수 있음

In [25]:
import random
distinctChars = words.flatMap(lambda word:list(word.lower())).distinct()

charRDD = distinctChars.map(lambda c : (c, random.random()))
charRDD2 = distinctChars.map(lambda c : (c, random.random()))

charRDD.cogroup(charRDD2).map(lambda row : (row[0], list(row[1][0]), list(row[1][1]))).take(5)

[('s', [0.6180502779695917], [0.7259047289921635]),
 ('p', [0.17179780996984817], [0.10066614711489186]),
 ('r', [0.2768817952346454], [0.4716119523279535]),
 ('i', [0.250440715152488], [0.6046769419701608]),
 ('g', [0.30498268844803544], [0.4212155835062287])]

## 13.4 조인
구조적 API와 모두 동일한 기본 형식을 사용
- 출력 파티션 수나 사용자 정의 파티션 함수를 파라미터로 사용

### 내부 조인
다른 조인도 마찬가지로 적용 by key

In [32]:
keyedChars = distinctChars.map(lambda c : (c, random.random()))
outputPartitions = 10

KVcharacters.join(keyedChars).collect()
KVcharacters.join(keyedChars, outputPartitions).collect()

[('u', (1, 0.16164537283386482)),
 ('m', (1, 0.7259047289921635)),
 ('m', (1, 0.7259047289921635)),
 ('k', (1, 0.17179780996984817)),
 ('t', (1, 0.2768817952346454)),
 ('t', (1, 0.2768817952346454)),
 ('t', (1, 0.2768817952346454)),
 ('e', (1, 0.9808139104191489)),
 ('e', (1, 0.9808139104191489)),
 ('e', (1, 0.9808139104191489)),
 ('e', (1, 0.9808139104191489)),
 ('e', (1, 0.9808139104191489)),
 ('e', (1, 0.9808139104191489)),
 ('e', (1, 0.9808139104191489)),
 ('f', (1, 0.4874474039474913)),
 ('n', (1, 0.250440715152488)),
 ('n', (1, 0.250440715152488)),
 (':', (1, 0.2957658103767279)),
 ('o', (1, 0.16495429728330202)),
 ('s', (1, 0.9305358452181502)),
 ('s', (1, 0.9305358452181502)),
 ('s', (1, 0.9305358452181502)),
 ('s', (1, 0.9305358452181502)),
 ('p', (1, 0.07236554685680652)),
 ('p', (1, 0.07236554685680652)),
 ('p', (1, 0.07236554685680652)),
 ('r', (1, 0.698502468661265)),
 ('r', (1, 0.698502468661265)),
 ('h', (1, 0.7883321366844236)),
 ('i', (1, 0.9282857187616657)),
 ('i', (

### zip
동일한 길이의 2개의 RDD를 연결하여 PairRDD를 생성 >> 동일한 수의 요소와 파티션 필요
<br>기존 파이썬의 문법과 동일~

In [33]:
numRange = sc.parallelize(range(10), 2)
words.zip(numRange).collect()

[('Spark', 0),
 ('The', 1),
 ('Definitive', 2),
 ('Guide', 3),
 (':', 4),
 ('Big', 5),
 ('Data', 6),
 ('Processing', 7),
 ('Made', 8),
 ('Simple', 9)]

## 13.5 파티션 제어
RDD의 파티셔닝은 물리적으로 정확히 분산되는 방식을 정의 가능

### coalesce
파티션을 재분배할 때 발생하는 셔플을 방지하기 위해 동일한 워커에 존재하는 파티션을 합치는 메서드

In [38]:
words.coalesce(1).getNumPartitions()

1

### repartition
파티션 수를 조절하는 메서드 ; 파티션 수가 많아지면 병렬 처리 수준이 향상
<br>처리시 데이터간의 셔플이 발생할 수도...
- repartitionAndSortWithinPartitions 메서드
    - 파티션 재분배와 더불어 정렬방식을 지정가능

In [40]:
words.repartition(10).getNumPartitions()

10

### 사용자 정의 파티셔닝
파티셔닝의 목적은 데이터를 균등하게 배분하는것!!!
- 사용자 정의 파티셔닝을 사용하려면 Partitioner을 확장한 클래스를 구현

In [41]:
df = spark.read.option("header", "true").option("inferSchema", "true")\
.csv("file:///home/ubuntu/ybigta/Dataset_spark/data/retail-data/all/")
rdd = df.coalesce(10).rdd

df.printSchema()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: integer (nullable = true)
 |-- Country: string (nullable = true)



In [43]:
def partitionFunc(key):
    import random
    
    if key == 17850 or key == 12583:
        return 0
    else:
        return random.randint(1,2)
    
keyedRDD = rdd.keyBy(lambda x : x[6])

keyedRDD\
.partitionBy(3, partitionFunc)\
.map(lambda x : x[0])\
.glom()\
.map(lambda x : len(set(x)))\
.take(5)

[2, 4299, 4294]

## 13.6 사용자 정의 직렬화 (Kryo 직렬화)
그런게 있단다... 아직은 이해가 되질 않는다...
4부에서 공부해보자,.ㅜㅠㅜㅠ